In [13]:
# import required modules
import numpy as np
import cv2
from random import *

In [14]:
# initialise required parameters
rows = 500
cols = 500
nrows = 10
ncols = 10
reveal = False
bee = 9                 # bee or mines is present
nobee = 11              # bee or mines is absent
mouse_point = (-1,-1)   
h = int(rows / nrows)
w = int(cols / ncols)
grid = np.ones((nrows, ncols))*nobee       #initailise grid with no bees or mines
field = np.ones((rows,cols,3),np.uint8)*255
font = cv2.FONT_HERSHEY_SIMPLEX
pressed = False
clicked = []                        #  for tracking points that are clicked

In [15]:
def click(event, x, y, flags, param):
    global mouse_point, pressed
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_point = (x,y)
        pressed = True

cv2.namedWindow('field display')
cv2.setMouseCallback('field display', click)

class cell:
    def __init__(self,grid,field):
        self._grid = grid
        self._field = field
    def create_grid_with_bees(self):
        count_bee = 0
        while count_bee <10:
            i = randint(0,9)
            j = randint(0,9)
            grid[i][j] = bee
            count_bee += 1
        print(self._grid)    
        return count_bee
    
    def reveal_0(self,i,j):
        for xoff in range(-1,2):
            for yoff in range(-1,2):
                i1 = i + xoff
                j1 = j + yoff
            
                if i1>-1 and i1 < nrows and j1 > -1 and j1 < ncols and (i1,j1) not in clicked:
                    clicked.append((i1,j1))
                    cv2.rectangle(field,(j*w+1,i*h+1),(j*w+w-1,i*h+h-1),(128,128,128),-1)
                    cv2.rectangle(field,(j1*w+1,i1*h+1),(j1*w+w-1,i1*h+h-1),(128,128,128),-1)
                    if self.count(i1,j1) != 0:
                        cv2.rectangle(field,(j1*w+1,i1*h+1),(j1*w+w-1,i1*h+h-1),(200,200,200),-1)
                        cv2.putText(field,str(self.count(i1,j1)),(j1*w+int(w/2)-8,i1*h+int(h/2)+10), font, 0.8,(0,0,0),1,cv2.LINE_AA)
                    else:
                        self.reveal_0(i1,j1)
        return
    
    def count(self,i,j):
        coun = 0
        for xoff in range(-1,2):
            for yoff in range(-1,2):
                i1 = i + xoff
                j1 = j + yoff
                if i1>-1 and i1 < nrows and j1 > -1 and j1 < ncols:
                    if self._grid[i1][j1] == bee:
                        coun = coun + 1
        return coun
    
    def show(self):
        if reveal == True:
            for i in range(0,rows,h):
                for j in range(0,cols,w):
                    cv2.rectangle(field,(j+1,i+1),(j+w-1,i+h-1),(200,200,200),-1)
                    if self._grid[i//h][j//w] == bee:
                        cv2.circle(self._field,(j+int(h/2),i+int(w/2)),int(0.3*w),(0,0,0),-1)
                    else:
                        coun = self.count(i//h,j//w)
                    
                        if coun !=0:
                            cv2.putText(self._field,str(coun),(j+int(w/2)-8,i+int(h/2)+10), font, 0.8,(0,0,0),1,cv2.LINE_AA)
                        
        else:
            return
    

In [16]:
cel = cell(grid,field)
count_bee = cel.create_grid_with_bees()

for i in range(0,rows,w):
    for j in range(0,cols,h):
        cv2.rectangle(field,(j,i),(j+h,i+w),(20,20,20),1)

while(True):
    
    if pressed == True and len(clicked) != nrows*ncols - count_bee and reveal == False: 
        j= mouse_point[0]//w
        i= mouse_point[1]//h
        if (i,j) not in clicked:
            clicked.append((i,j))
            if grid[i][j] == bee:
                reveal = True
                cel.show()
                cv2.putText(field,'You Lose',(int(rows/2)-100,int(cols/2)), font, 2,(0,0,0),4,cv2.LINE_AA)
            else:
                coun = cel.count(i,j)
                if coun==0:
                    cel.reveal_0(i,j)
                else:
                    cv2.rectangle(field,(j*w+1,i*h+1),(j*w+w-1,i*h+h-1),(200,200,200),-1)
                    cv2.putText(field,str(coun),(j*w+int(w/2)-8,i*h+int(h/2)+10), font, 0.8,(0,0,0),1,cv2.LINE_AA)
                    
        pressed = False
    
    cv2.imshow('field display', field)    
    if len(clicked) == nrows*ncols - count_bee:
        cv2.putText(field,'You Win',(int(rows/2)-100,int(cols/2)), font, 2,(0,0,0),4,cv2.LINE_AA)
         
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()


[[11. 11. 11. 11. 11. 11. 11. 11. 11. 11.]
 [ 9. 11. 11. 11. 11. 11. 11. 11. 11. 11.]
 [11.  9.  9. 11. 11. 11. 11. 11. 11.  9.]
 [11. 11. 11. 11. 11. 11. 11. 11. 11. 11.]
 [11. 11. 11. 11.  9. 11. 11. 11. 11. 11.]
 [ 9. 11. 11. 11. 11. 11. 11. 11. 11. 11.]
 [11. 11. 11. 11.  9. 11. 11. 11. 11. 11.]
 [11.  9. 11.  9. 11. 11.  9. 11. 11. 11.]
 [11. 11. 11. 11. 11. 11. 11. 11. 11. 11.]
 [11. 11. 11. 11. 11. 11. 11. 11. 11. 11.]]
